# Project 3: Activity Classification with Various Classifiers

- 제출기한: 2020년 9월 4일 금요일 오후 11시 59분 59초까지
- 제출방법 - 이메일 (여러 번 제출할 경우 마지막 제출 사용)
  - 메일주소: `hcs.lab.2019@gmail.com`
  - 메일제목: `Group0 ML project 3` (Group1, Group2, ... 식으로 Group명을 바꿔주세요)
  - 제출 내용: `project_3.ipynb` 의 [3번 Cell](#Various-Classifiers-with-Feature-Extractor-and-Preprocessor) (Various Classifiers with Feature Extractor and Preprocessor) 의 코드를 이메일 본문에 삽입해서 제출
    - **%%%%% 주의사항 %%%%%**
    - **3번 Cell을 제외한 다른 부분의 코드는 제출하지 않고, 채점도 되지 않습니다.**
    - **3번 Cell의 `lr_clf`, `lda_clf`, `rf_clf`, `svm_clf`, `boost_clf` 변수들의 이름을 수정하시면 안됩니다. (채점시 해당 변수들을 Classifier로 사용 예정)**
  - 채점방식은 가장 아래의 [Scoring](#Scoring) 부분을 참고해주세요

- 과제 목표: 여러 가지 종류의 Classifier를 이용한 Activity Recognition
- 과제 방식
  1. 여러 가지 종류의 Classifier를 만듭니다.
    - `Logistic Regression`, `Linear Discriminant Analysis`, `Random Forest`, `Support Vector Machines`, `Boosting with Decision Trees` 의 5가지 방식을 사용합니다. 각각 `lr_clf`, `lda_clf`, `rf_clf`, `svm_clf`, `boost_clf` 변수에 해당하고, **각 변수는 해당하는 방식으로 최종 Classification을 진행해야합니다.**
  2. 각 Classifier에 대해 Cross Validation을 이용하여 Test macro f1 score가 가장 높게 나올만한 Feature, Preprocessor, Hyperparameter를 구합니다.
  4. Pipeline을 이용하여 FeatureExtracting과 Preprocessing, Classification을 해주는 Classifier들을 만들어 제출합니다. (채점시 `lr_clf`, `lda_clf`, `rf_clf`, `svm_clf`, `boost_clf` 변수들을 Classifier로 사용 예정)
- 채점 기준
  - 제출한 3번 Cell의 5가지 분류기들을 이용하여 각각 train data를 학습시키고, test data에 대해 macro f1 score를 측정합니다.
  - train data는 train.pkl로 주어진 7명의 데이터이고, test data는 주어지지 않은 2명의 데이터입니다.
  - f1 score는 precision과 recall의 조화평균입니다.
  - f1 score는 각 Motion마다 구해집니다. 전체 데이터에 대한 f1 score를 구하는 방법에는 크게 micro, macro, weighted 3가지 방식이 있습니다.
    - micro f1 score: 전체 데이터에 대한 precision과 recall의 조화평균으로 accuracy와 같습니다.
    - macro f1 score: 각 Motion별 f1 score의 평균
    - weighted f1 score: 각 Motion의 sample수에 비례하게 weight를 준 평균
  - project_3의 경우 5가지 분류기의 macro f1 score의 평균값을 이용해 평가합니다.
  - 최고 score 팀은 16점, 최저 score 팀은 11점이고, 그 사이의 팀들은 score에 비례하여 점수를 매깁니다.
  - (ex. 3팀, 5가지 분류기의 평균 macro f1 score: [0.7, 0.6, 0.5] => 점수: [16점, 13.5점, 11점])

---
## Input Data Description
<img src="example.png" width='500px'>
<div style='text-align:center'>Figure 1. Data Example (2 samples)</div><br/>
<div>1. Figure 1과 같이 손목에 달려있는 가속도계의 x, y, z축 가속도 정보를 이용하여 행동을 예측한다.</div>
<div>2. 주어진 Train Data(data/train.pkl)는 총 7명의 데이터이다. (총 13940개의 데이터 샘플)</div>
<div>3. 각 데이터 샘플은 Subject, Data, Motion으로 이루어지고 의미는 아래와 같다.</div>
<div style='text-indent:20px'>Subject: 샘플을 수집한 실험자의 id, type은 int (분류할 때 feature (predictor) 로 이용하지 않는다)</div>
<div style='text-indent:20px'>Data: (500, 3) Shape의 Numpy array (1초당 100개씩 5초간 x, y, z축의 가속도값을 측정)</div>
<div style='text-indent:20px'>Motion: 행동의 종류, type은 string (ex. 'walking')</div>
<div>4. Subject 정보는 분류에 이용되지 않는다.</div>
<div>5. Subject 정보를 이용하여 train data와 validation data를 나눌 수 있다.</div>

### Data Load

In [16]:
# 아래와 같은 방식으로 train data를 load할 수 있다.
import pickle

with open('data/train.pkl', 'rb') as f:
    df_train = pickle.load(f)
df_train.head(3)

,Subject,Data,Motion
0,3,"[[-2.115489959716797, 2.9105000495910645, 8.15...",ironing
1,3,"[[-3.5734200477600098, 3.9679501056671143, 10....",ironing
2,3,"[[-1.7668700218200684, 11.98840045928955, 1.08...",ironing


In [17]:
# 9가 하나의 Group을 담당하기엔 너무 작은 것 같다. 7로 변경하자
df_train.loc[df_train['Subject'] == 9, 'Subject'] = 7
df_train

,Subject,Data,Motion
0,3,"[[-2.115489959716797, 2.9105000495910645, 8.15...",ironing
1,3,"[[-3.5734200477600098, 3.9679501056671143, 10....",ironing
2,3,"[[-1.7668700218200684, 11.98840045928955, 1.08...",ironing
3,3,"[[-4.2224202156066895, 9.517740249633789, 2.67...",ironing
4,3,"[[-6.610869884490967, 4.498380184173584, 7.815...",ironing
...,...,...,...
13935,7,"[[-1.8887399435043335, 2.340670108795166, 8.27...",rope jumping
13936,7,"[[-5.959249973297119, -7.587969779968262, 1.86...",rope jumping
13937,7,"[[4.046329975128174, 14.204400062561035, -0.56...",rope jumping
13938,7,"[[-14.82509994506836, 37.050201416015625, -8.1...",rope jumping


### Data Summary

In [18]:
print(f'Data 종류는 {list(df_train.columns)}가 있습니다')
print()

subjects, X, y = df_train.Subject, df_train.Data, df_train.Motion
print(f'Subject 개수: {len(subjects)}')
print(f'X 개수: {len(X)}')
print(f'y 개수: {len(y)}')
print()

print('Subject는 int, Data는 numpy.ndarray, Motion은 string 값을 가집니다')
print(f'type(subjects[0]): {type(subjects[0])}')
print(f'type(X[0]): {type(X[0])}')
print(f'type(y[0]): {type(y[0])}')
print()

print(f'X는 {X[0].shape} 형태의 np.ndarray입니다')
print()

print(f'y (motion) classes: {sorted(list(set(y)))}')
print()

print(f'Subject들: {set(subjects)}')
print()

Data 종류는 ['Subject', 'Data', 'Motion']가 있습니다

Subject 개수: 13940
X 개수: 13940
y 개수: 13940

Subject는 int, Data는 numpy.ndarray, Motion은 string 값을 가집니다
type(subjects[0]): <class 'numpy.int64'>
type(X[0]): <class 'numpy.ndarray'>
type(y[0]): <class 'numpy.str_'>

X는 (500, 3) 형태의 np.ndarray입니다

y (motion) classes: ['Nordic walking', 'ascending stairs', 'cycling', 'descending stairs', 'ironing', 'lying', 'rope jumping', 'running', 'sitting', 'standing', 'vacuum cleaning', 'walking']

Subject들: {3, 4, 5, 6, 7, 8}



In [25]:
# ---------------------------------------------------------------------------------------------------------------------
#  Logistic Regression, Linear Discriminant Analysis, Random Forest, Support Vector Machines, Boosting with Decision Trees
#  위 5가지 방식으로 동작하는 Classifier를 만들어주세요.
#  (각각 변수명은 `lr_clf`, `lda_clf`, `rf_clf`, `svm_clf`, `boost_clf`입니다.)
# ---------------------------------------------------------------------------------------------------------------------
#  1. project_2.ipynb를 참고하시면 도움이 될 것 같습니다.
#
#  2. 각 Classifier마다 서로 다른 Feature, Preprocessor를 사용하셔도 됩니다.
#    - project_3.ipynb 가장 아래 부분의 Scoring 파트가 잘 동작하고,
#    - 각 Classifier가 문제에서 제시된 알고리즘으로 분류를 한다면 나머지 부분은 원하는 방식으로 작성하시면 됩니다.
#      (ex. `lr_clf` 인데 내부 로직은 Random Forest를 사용한다거나 하시면 안됩니다. )
# ---------------------------------------------------------------------------------------------------------------------

import numpy as np
import pandas as pd
from scipy import fft, stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.svm import SVC


def preprocess(X): # Wrapper of extract_features
    '''
    Preprocess multiple samples
    input
        X : (n, 500, 3)
    output
        X_processed : (n, p)
    '''
    X_processed = []
    xyz = lambda f: [v + '__' + f for v in 'xyzm']

    for i, X_sample in enumerate(X):
        mag = np.sqrt(np.sum(X_sample ** 2, axis=1)).reshape(-1, 1)
        X_sample = np.hstack([X_sample, mag])
        
        extracted = []
        columns = []

        columns += xyz('std')
        extracted.append(X_sample.std(axis=0))

        columns += xyz('cid_ce_norm')
        s = np.std(X_sample, axis=0)
        x = (X_sample - np.mean(X_sample, axis=0)) / s
        diff = np.diff(x, axis=0)
        extracted.append(np.sqrt(np.sum(diff**2, axis=0)))

        columns += xyz('number_crossing_0')
        positive = X_sample > 0
        diff = np.diff(positive, axis=0)

        count = [
            np.where(diff[:, 0])[0].size,
            np.where(diff[:, 1])[0].size,
            np.where(diff[:, 2])[0].size,
            np.where(diff[:, 3])[0].size,
        ]
        extracted.append(count)

        columns += xyz('number_crossing_mean')
        positive = X_sample > np.mean(X_sample, axis=0)
        diff = np.diff(positive, axis=0)
        count = [
            np.where(diff[:, 0])[0].size,
            np.where(diff[:, 1])[0].size,
            np.where(diff[:, 2])[0].size,
            np.where(diff[:, 3])[0].size,
        ]
        extracted.append(count)

        # quantiles 
        for p in [0.1, 0.5, 0.9]: 
            columns += xyz('percentile_' + str(p))
            extracted.append(np.quantile(X_sample, p, axis=0))

        columns += xyz('count_above_mean')
        extracted.append(np.sum(X_sample > np.mean(X_sample, axis=0), axis=0))

        columns += xyz('count_above_0')
        extracted.append(np.sum(X_sample > 0, axis=0))

        columns += xyz('abs_energy')
        extracted.append((X_sample ** 2).sum(axis=0))

        columns += xyz('absolute_sum_of_changes')
        extracted.append(np.sum(np.abs(np.diff(X_sample, axis=0)), axis=0))

        fft_abs = np.abs(fft.rfft(X_sample, axis=0))[1:]

        columns += xyz('fft_energy')
        extracted.append((fft_abs ** 2).sum(axis=0))

        columns += xyz('fft_centroid')
        extracted.append(np.sum(fft_abs * np.arange(len(fft_abs)).reshape(-1, 1), axis=0) / np.sum(fft_abs, axis=0))

        columns += xyz('fft_variance')
        moment2 = np.sum(fft_abs * np.arange(len(fft_abs)).reshape(-1, 1) ** 2, axis=0) / np.sum(fft_abs, axis=0)
        centroids = np.sum(fft_abs * np.arange(len(fft_abs)).reshape(-1, 1), axis=0) / np.sum(fft_abs, axis=0)
        fft_var = moment2 - centroids ** 2
        extracted.append(fft_var)

        columns += xyz('spectral_entropy')
        extracted.append(stats.entropy(fft_abs))

        X_processed.append(np.hstack(extracted))

    df = pd.DataFrame(data=np.array(X_processed), columns=columns)

    return df


lr_clf = Pipeline([  # Classifier using Logistic Regression
    ('feature_extractor', FunctionTransformer(preprocess)),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=250, solver='newton-cg', C=0.01)),
])

lda_clf = Pipeline([  # Classifier using Linear Discriminant Analysis
    ('feature_extractor', FunctionTransformer(preprocess)),
    ('scaler', StandardScaler()),
    ('classifier', LinearDiscriminantAnalysis(solver='svd')),
])

rf_clf = Pipeline([  # Classifier using Random Forest
    ('feature_extractor', FunctionTransformer(preprocess)),
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=400, criterion="gini", ccp_alpha=0.0, max_depth=24)),
])

svm_clf = Pipeline([  # Classifier using Support Vector Machines
    ('feature_extractor', FunctionTransformer(preprocess)),
    ('scaler', StandardScaler()),
    ('classifier', SVC(C=0.6, kernel='rbf')),
])

boost_clf = Pipeline([  # Classifier using Boosting with Decision Tree
    ('feature_extractor', FunctionTransformer(preprocess)),
    ('scaler', StandardScaler()),
    ('classifier', AdaBoostClassifier(
        base_estimator=DecisionTreeClassifier(max_depth=16), 
        learning_rate=1.0, 
        n_estimators=512)
    ),
])

## Cross Validation

In [26]:
clf = lr_clf  # Set classifier to tune

In [27]:
from sklearn.model_selection import cross_validate, LeaveOneGroupOut

clf_list = [lr_clf, lda_clf, rf_clf, svm_clf, boost_clf]
#mask = subjects != 9
#X, y, subjects = X[mask], y[mask], subjects[mask]
for clf in clf_list:
    cv_result = cross_validate(clf, X, y, groups=subjects, scoring='f1_macro', cv=LeaveOneGroupOut())
    average_test_score = np.mean(cv_result['test_score'])
    print(f'Avarage macro f1 score: {average_test_score}')

Avarage macro f1 score: 0.7414417359550974
Avarage macro f1 score: 0.7761027800627894
Avarage macro f1 score: 0.7258576125317971
Avarage macro f1 score: 0.7160844646811767
Avarage macro f1 score: 0.7428753963852621


## GridSearchCV

In [6]:
clf = rf_clf  # Set classifier to tune

In [7]:
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut

param_grid = {
    'classifier__criterion': ['gini', 'entropy'],  # Example param_grid for rf_clf
}

grid_search_cv = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=LeaveOneGroupOut(),
    scoring='f1_macro'
)

grid_search_cv.fit(X, y, groups=subjects)

print(f'Best params: {grid_search_cv.best_params_}')

Best params: {'classifier__criterion': 'entropy'}


---
# Scoring
- 아래 코드들은 실제 채점시 사용될 코드입니다.
  - 제출하신 Cell의 코드를 `project_3.ipynb`에 덮어쓰고 Restart & Run All을 실행하여 나온 결과를 사용합니다.
- `test.pkl`에는 `train.pkl`에 없는 다른 2명의 subject 데이터가 들어있습니다.
- 프로젝트를 하실 때에는 `test.pkl`이 없으므로 아래 코드가 실행되지 않는게 정상입니다.
- 채점은 아래 코드로 이루어진다는 것을 참고하시면 좋을 것 같습니다.

In [8]:
import pickle

with open('data/train.pkl', 'rb') as f:
    df_train = pickle.load(f)

with open('data/test.pkl', 'rb') as f:
    df_test = pickle.load(f)
df_test.head(3)

,Data,Motion
0,"[[2.43859, 7.02674, 5.74905], [2.58814, 6.9873...",ironing
1,"[[2.52346, 6.31869, 7.25372], [2.02206, 5.2595...",ironing
2,"[[-3.03257, 8.39546, 1.27492], [-3.17651, 8.28...",ironing


In [9]:
X_train, X_test, y_train, y_test = df_train.Data, df_test.Data, df_train.Motion, df_test.Motion
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(13940,) (5007,) (13940,) (5007,)


In [ ]:
from sklearn.metrics import f1_score

f1_scores = []
clfs = [lr_clf, lda_clf, rf_clf, svm_clf, boost_clf]
for clf in clfs:
    y_pred = clf.fit(X_train, y_train).predict(X_test)
    f1_scores.append(f1_score(y_test, y_pred, average='macro'))

In [ ]:
print(np.mean(f1_scores))